In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import os
import strategy_simulator as ss

### Load and process data

In [ ]:
price_data = pd.read_csv(os.path.join('..', 'data', 'scraped_OHLC', 'BTCUSD.csv'))
price_data['dtime'] = pd.to_datetime(price_data['dtime'], format='%Y-%m-%d %H:%M:%S')
price_data.set_index('dtime', inplace = True)
price_data.dropna(inplace = True)

# price_data = pd.read_pickle(os.path.join('..', 'data', 'SOL', "SOLUSD_price-volume_minute.pickle"))
# price_data

price_data 

In [ ]:
plt.plot(price_data['open'])
plt.show()
price_data_hourly = price_data.resample('1H').mean().fillna(method='ffill')
plt.plot(price_data_hourly['open'])
plt.show()

In [ ]:
x_new = [price_data_hourly['open'].iloc[0]]
for i in range(len(price_data_hourly) - 1):
    price = price_data_hourly['open'].iloc[i+1]
    x_new.append(ss.exponential_moving_average(price, x_new[-1], 0.05))

price_data_hourly['mean'] = x_new
fig = px.line(price_data_hourly[-1000:], y=['open', 'mean'])
fig.show()



### Test strategies using trade simulator

In [ ]:
last_n = len(price_data_hourly)


s1 = ss.hodl()
s2 = ss.DCA(period = last_n/10, amount = 10)
s3 = ss.randomDCA(period = last_n/10, amount = 10)
s4 = ss.percentage_from_mean(avg_start = price_data_hourly['open'].iloc[-last_n])
s5 = ss.maintain_investment()
plt.plot(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
plt.show()

strategies = [s1, s2, s3, s4, s5]

for s in strategies:
    sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
    sim.run(s)
    plt.plot(sim.invested_ot)
    plt.show()

In [ ]:
last_n = 2000 #len(resampled)
strategy = ss.maintain_investment()
sim = sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
sim.run(strategy)
    

plt.plot(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
plt.show()
plt.plot(sim.invested_ot)
# plt.plot(sim.reserves)
plt.show()

In [ ]:
last_n = len(price_data_hourly)
strategy = ss.slow_fast_avg(avg_start = price_data_hourly['open'].iloc[-last_n], beta_slow = 0.1, beta_fast = 0.3, trade_amount = 1000)
sim = sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
sim.run(strategy)
    

# plt.plot(price_data_hourly[-last_n:]['price'].reset_index(drop = True))
# plt.plot(strategy.avg_fast_ot)
# plt.plot(strategy.avg_slow_ot)
# plt.show()
# plt.plot(sim.invested_ot)
# # plt.plot(sim.reserves)
# plt.show()

temp = price_data_hourly[-last_n:]
temp['fast_avg'] = strategy.avg_fast_ot
temp['slow_avg'] = strategy.avg_slow_ot
temp['trade'] = (sim.invested_ot > np.mean(sim.invested_ot)) *1000 + temp['open'].mean()
fig = px.line(temp, y=['open', 'fast_avg', 'slow_avg', 'trade'])
fig.show()


In [ ]:
last_n = len(price_data_hourly)
# strategy = ss.slow_fast_avg(avg_start = price_data_hourly['price'].iloc[-last_n], beta_slow = 0.1, beta_fast = 0.3, trade_amount = 1000)
strategy = ss.avg_derivitive(avg_start = price_data_hourly['open'].iloc[-last_n], beta = 0.1)
sim = sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
sim.run(strategy)

temp = price_data_hourly[-last_n:]
temp['avg'] = strategy.avg_ot
temp['invested'] = sim.invested_ot
fig = px.line(temp, y=['open', 'avg'])
fig.show()
fig = px.line(temp, y=['invested'])
fig.show()


In [ ]:
last_n = int(len(price_data_hourly))
# strategy = ss.slow_fast_avg(avg_start = price_data_hourly['price'].iloc[-last_n], beta_slow = 0.1, beta_fast = 0.3, trade_amount = 1000)
strategy = ss.opportunistic(avg_start = price_data_hourly['open'].iloc[-last_n], beta = 0.05, thresh_percent = .05, trade_amount = 1)
sim = sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
sim.run(strategy)

temp = price_data_hourly[-last_n:]
temp['avg'] = strategy.avg_ot
temp['invested'] = sim.invested_ot
fig = px.line(temp, y=['open', 'avg'])
fig.show()
fig = px.line(temp, y=['invested'])
fig.show()



In [ ]:
last_n = len(price_data_hourly)
# strategy = ss.slow_fast_avg(avg_start = price_data_hourly['price'].iloc[-last_n], beta_slow = 0.1, beta_fast = 0.3, trade_amount = 1000)
strategy = ss.momentum(trade_amount = .1,trade_cap=1 )
sim = sim = ss.trade_simulator(price_data_hourly[-last_n:]['open'].reset_index(drop = True))
sim.run(strategy)

temp = price_data_hourly[-last_n:]
temp['invested'] = sim.invested_ot
fig = px.line(temp, y=['open'])
fig.show()
fig = px.line(temp, y=['invested'])
fig.show()